# Fundamentos de Robótica
## Exercício 5
### Danilo Calhes Paixão dos Santos - 11025215

Foram definidos os eixos na configuração apresentada abaixo:

<img src=./Exercicio_5/img/robo.jpg>
<img src=./Exercicio_5/img/robo_eixos.jpg>

Dessa maneira, determinou-se os seguintes parâmetros na tabela de Denavit-Hartenberg:

|Junta| a | alpha | d   | theta |
|---|---|-------|-----|-------|
| 1 | 0 | -90   | 450 | $^*$$\theta_1$     |
| 2 | 0 | 90    | 200 | $^*$$\theta_2$   |
| 3 | 0 | 0     |  $^*$d$_3$ | 0     |
| 4 | 0 | -90   | 150 | $^*$$\theta_4$    |
| 5 | 0 | -90   | 100 | $^*$$\theta_5$     |
| 6 | 0 | 0     | 75  | $^*$$\theta_6$     |

E dessa maneira, determina-se por meio da matriz homogênea($A^i_{i-1}$) as matrizes homogeneas relativas a configuração de juntas ($q$).

$$q = \begin{bmatrix}
-45^\mathrm{o}\\ 
0^\mathrm{o}\\ 
-100mm\\ 
-90^\mathrm{o}\\ 
90^\mathrm{o}\\
90^\mathrm{o}
\end{bmatrix}$$

$$A^i_{i-1} = Rot_z(\theta)Trans_z(d)Trans_x(a)Rot_x(\alpha)$$

$$A^i_{i-1} = \begin{bmatrix}
 c_{\theta_i} & -s_{\theta_i}c_{\alpha_i} & s_{\theta_i}s_{\alpha_i} &  a_i c_{\theta_i}\\ 
 s_{\theta_i} &  c_{\theta_i}c_{\alpha_i}& -c_{\theta_i}s_{\alpha_i} & a_i s_{\theta_i}\\ 
 0 & s_{\alpha_i} & c_{\alpha_i} & d_i\\ 
 0 & 0  & 0 & 1
\end{bmatrix}$$

Para determina-las, criam-se algumas funções:

In [10]:
import numpy as np
import pandas as pd
import os
def getElementarMatrix(a,alpha,d,theta):
    c_theta = np.cos(np.radians(theta))
    s_theta = np.sin(np.radians(theta))
    c_alpha = np.cos(np.radians(alpha))
    s_alpha = np.sin(np.radians(alpha))

    matrix = np.array([
        [c_theta,-s_theta*c_alpha,s_theta*s_alpha,a*c_theta],
        [s_theta,c_theta*c_alpha,-c_theta*s_alpha,a*s_theta],
        [0,s_alpha,c_alpha,d],
        [0,0,0,1]
    ])

    return matrix

def getConfiguration(dh_mat,folder='results',name='mat',convert=[''],verbose=True,save=False):
    if(verbose): printDHmatrix(dh_mat)
    if(len(convert)>0):
        convertDHmatrix(dh_mat,folder=folder,name=name,types=convert,save=save)
    
    joints = dh_mat.shape[0]
    if(verbose): print("+==== No. of joints: ",joints)
    configs = []
    for joint,props in enumerate(dh_mat):
        if(verbose): print("+== Joint {}: {}".format(joint+1,props))
        if(verbose): print("A_0{}:\n{}\n".format(joint+1,getElementarMatrix(*np.asarray(props))))
        if(len(convert)>0):
            convertMatrix(dh_mat,folder=folder,name=name+str(joint+1),types=convert,save=save)
        configs.append({"joint":joint+1,
                       "matrix":getElementarMatrix(*np.asarray(props))})
    return configs
        
def printDHmatrix(dh_matrix):
    DF_mat = pd.DataFrame(dh_matrix, index=range(1,len(dh_matrix)+1), columns=["a","alpha","d","theta"])
    print(DF_mat,"\n")
    
def convertDHmatrix(dh_matrix,verbose=False,types=['latex', 'csv'],folder="",name="mat",save=False):
    if 'latex' in types:
        DF_mat = pd.DataFrame(dh_matrix, index=range(1,len(dh_matrix)+1), columns=["a","alpha","d","theta"])
        latex_output = DF_mat.to_latex()
        if(verbose): print(latex_output)
        if(save):
            filename = open(os.path.join(".",folder,name+"_DHmat.latex"),'w')
            filename.write(latex_output)
            filename.close()
        
    if 'csv' in types:
        DF_mat = pd.DataFrame(dh_matrix, index=range(1,len(dh_matrix)+1), columns=["a","alpha","d","theta"])
        csv_output = DF_mat.to_csv()
        if(verbose): print(csv_output)
        if(save):
            filename = open(os.path.join(".",folder,name+"_DHmat.csv"),'w')
            filename.write(csv_output)
            filename.close()
   # Para python>=3.6.1 com pandas==1.0.1     
   # if 'md' in types:
        #DF_mat = pd.DataFrame(dh_matrix)
        #if(verbose): print(DF_mat.to_markdown())
            
def convertMatrix(dh_matrix,verbose=False,types=['latex', 'csv'],folder="",name="mat",save=False):
    if 'latex' in types:
        DF_mat = pd.DataFrame(dh_matrix)
        latex_output = DF_mat.to_latex()
        if(verbose): print(latex_output)
        if(save):
            filename = open(os.path.join(".",folder,name+"_mat.latex"),'w')
            filename.write(latex_output)
            filename.close()
        
    if 'csv' in types:
        DF_mat = pd.DataFrame(dh_matrix)
        csv_output = DF_mat.to_csv(index=False,header=False) 
        if(verbose):  print(csv_output)
        if(save):
            filename = open(os.path.join(".",folder,name+"_mat.csv"),'w')
            filename.write(csv_output)
            filename.close()
    # Para python>=3.6.1 com pandas==1.0.1    
    #if 'md' in types:
        #DF_mat = pd.DataFrame(dh_matrix)
        #if(verbose): print(DF_mat.to_markdown())

Após criadas as funções, utiliza-se para determinar a configuração de cada uma das articulações:

In [11]:
def getDHmatrix(theta_1=-45,
                theta_2=0,
                d_3=-100,
                theta_4=-90,
                theta_5=90,
                theta_6=90):
    dh_mat=np.array([
        [0 , -90 , 450 , theta_1 ],
        [0 , 90 , 200 , theta_2 ],
        [0 , 0 , d_3 , 0 ],
        [0 , -90 , 150 , theta_4 ],
        [0 , -90 , 100 , theta_5 ],
        [0 , 0 , 75 , theta_6 ]
    ])
    
    return dh_mat
    
    

config_1={"theta_1":-45.,
          "theta_2":0.,
          "d_3":-100.,
          "theta_4":-90.,
          "theta_5":90.,
          "theta_6":90.}

dh_matrix= getDHmatrix(**config_1)
configurations = getConfiguration(dh_matrix,folder="Exercicio_5/results/", convert=['latex','csv'],save=True,verbose=True)

     a  alpha      d  theta
1  0.0  -90.0  450.0  -45.0
2  0.0   90.0  200.0    0.0
3  0.0    0.0 -100.0    0.0
4  0.0  -90.0  150.0  -90.0
5  0.0  -90.0  100.0   90.0
6  0.0    0.0   75.0   90.0 

+==== No. of joints:  6
+== Joint 1: [  0. -90. 450. -45.]
A_01:
[[ 7.07106781e-01  4.32978028e-17  7.07106781e-01  0.00000000e+00]
 [-7.07106781e-01  4.32978028e-17  7.07106781e-01 -0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  6.12323400e-17  4.50000000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

+== Joint 2: [  0.  90. 200.   0.]
A_02:
[[ 1.000000e+00 -0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00  0.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17  2.000000e+02]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]

+== Joint 3: [   0.    0. -100.    0.]
A_03:
[[   1.   -0.    0.    0.]
 [   0.    1.   -0.    0.]
 [   0.    0.    1. -100.]
 [   0.    0.    0.    1.]]

+== Joint 4: [  0. -90. 150. -90.

Por fim, a configuração é dada pela matriz:

$$A^6_{0} = \begin{bmatrix}
 0 & 1 & 0 &  0\\ 
 1 & 0 & 0 & 0\\ 
 0 & 0 & 1 & 75.\\ 
 0 & 0  & 0 & 1
\end{bmatrix}$$

### Errata (27/03)
O primeiro erro se deu na definição da matriz elementar. Ao passar como argumento para a função criada, o argumento foi passado em graus, onde deveria ser passado em radianos:

In [ ]:
(...)
def getElementarMatrix(a,alpha,d,theta):
    c_theta = np.cos(theta)
    s_theta = np.sin(theta)
    c_alpha = np.cos(alpha)
    s_alpha = np.sin(alpha)
(...)

Dessa maneira corrigiu-se o padrão da função para graus:

In [ ]:
(...)
def getElementarMatrix(a,alpha,d,theta):
    c_theta = np.cos(np.radians(theta))
    s_theta = np.sin(np.radians(theta))
    c_alpha = np.cos(np.radians(alpha))
    s_alpha = np.sin(np.radians(alpha))
(...)

Além disso, a solução obtida no exercício apresentava apenas as matrizes de transformação e não a matriz homogenea do efetuador, dessa maneira adiciona-se uma etapa extra:

In [14]:
homogen = np.identity(4) #inicializa a matriz identidade

for config in configurations:
    print("++ Junta ",config['joint'])
    homogen = homogen@config['matrix']
    with np.printoptions(precision=4, suppress=True):
        print(homogen)

++ Junta  1
[[  0.7071   0.       0.7071   0.    ]
 [ -0.7071   0.       0.7071   0.    ]
 [  0.      -1.       0.     450.    ]
 [  0.       0.       0.       1.    ]]
++ Junta  2
[[  0.7071   0.7071   0.     141.4214]
 [ -0.7071   0.7071  -0.     141.4214]
 [  0.       0.       1.     450.    ]
 [  0.       0.       0.       1.    ]]
++ Junta  3
[[  0.7071   0.7071   0.     141.4214]
 [ -0.7071   0.7071  -0.     141.4214]
 [  0.       0.       1.     350.    ]
 [  0.       0.       0.       1.    ]]
++ Junta  4
[[ -0.7071   0.       0.7071 141.4214]
 [ -0.7071  -0.      -0.7071 141.4214]
 [  0.      -1.       0.     500.    ]
 [  0.       0.       0.       1.    ]]
++ Junta  5
[[  0.      -0.7071   0.7071 212.132 ]
 [ -0.       0.7071   0.7071  70.7107]
 [ -1.      -0.      -0.     500.    ]
 [  0.       0.       0.       1.    ]]
++ Junta  6
[[ -0.7071  -0.       0.7071 265.165 ]
 [  0.7071   0.       0.7071 123.7437]
 [ -0.       1.      -0.     500.    ]
 [  0.       0.       0.  

Por fim a matriz homogênea que define a configuração da garra é dada por
$$H^6_{0} = \begin{bmatrix}
 -0.707 & 0 & 0.707 &  265.165\\ 
 0.707 & 0 &  0.707 & 123.744\\ 
 0 & 1. & 0 & 500.\\ 
 0 & 0  & 0 & 1
\end{bmatrix}$$